In [1]:
import sys

sys.path.append("../")

from datasets import load_dataset
from transformers import AutoTokenizer
from src.data import *
from accelerate import accelerator

In [2]:
dataset = RawMTDataset.wmt16_ro_en()
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenized_dataset = TokenizedMTDataset(dataset, tokenizer)

Reusing dataset wmt16 (/Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a)
100%|██████████| 3/3 [00:00<00:00, 57.78it/s]


In [7]:
# tokenizer("this is brazil")
tokenizer.vocab["[CLS]"]

101

In [30]:
preprocessed_dataset = tokenized_dataset.preprocess(8)

Loading cached processed dataset at /Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a/cache-31a41103e19cab6f.arrow
Loading cached processed dataset at /Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a/cache-65d66b5e8ff99eaa.arrow
Loading cached processed dataset at /Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a/cache-313bfa5a8bb711d4.arrow
Loading cached processed dataset at /Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a/cache-4e8c0a3acd489813.arrow
Loading cached processed dataset at /Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a/cache-faeee1de00e0c678.arrow
Loading cached processed datas

In [31]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-multilingual-cased")
model.add_adapter("testing")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
from transformers import BertLMHeadModel, BertConfig

config = BertConfig(vocab_size=119547, is_decoder=True, add_cross_attention=True)
model_lm = BertLMHeadModel.from_pretrained("bert-base-multilingual-cased", config=config)
model_lm.add_adapter("testing")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.4.crossattention.self.query.weight', 'bert.encoder.layer.6.crossattention.self.query.weight', 'bert.encoder.layer.10.crossattention.self.key.weight', 'bert.encoder.layer.7.crossattention.self.query

In [24]:
from transformers import EncoderDecoderModel
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-german-dbmdz-uncased', 'bert-base-uncased', )
# model1 = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-german-dbmdz-uncased', 'bert-base-uncased', )
# model.add_adapter("testing")
# model.train_adapter("testing")
for k, v in model.named_parameters():
    if "crossattention" in k or "decoder.cls.predictions" in k:
        v.requires_grad = True
    else:
        v.requires_grad = False

for k, v in model.named_parameters():
    print(k, v.requires_grad)
# for k, v in model.named_parameters():
#     if "crossattention" in k:
#         v.requires_grad = True
# count = 0
# for k, v in model.named_parameters():
#     if v.requires_grad:
#         print(k)
#         count += 1

# print(count)
# model.decoder.apply(model.encoder._init_weights)
# model.decoder.bert.embeddings.word_embeddings.weight, model1.decoder.bert.embeddings.word_embeddings.weight

Some weights of the model checkpoint at bert-base-german-dbmdz-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.

encoder.embeddings.word_embeddings.weight False
encoder.embeddings.position_embeddings.weight False
encoder.embeddings.token_type_embeddings.weight False
encoder.embeddings.LayerNorm.weight False
encoder.embeddings.LayerNorm.bias False
encoder.encoder.layer.0.attention.self.query.weight False
encoder.encoder.layer.0.attention.self.query.bias False
encoder.encoder.layer.0.attention.self.key.weight False
encoder.encoder.layer.0.attention.self.key.bias False
encoder.encoder.layer.0.attention.self.value.weight False
encoder.encoder.layer.0.attention.self.value.bias False
encoder.encoder.layer.0.attention.output.dense.weight False
encoder.encoder.layer.0.attention.output.dense.bias False
encoder.encoder.layer.0.attention.output.LayerNorm.weight False
encoder.encoder.layer.0.attention.output.LayerNorm.bias False
encoder.encoder.layer.0.intermediate.dense.weight False
encoder.encoder.layer.0.intermediate.dense.bias False
encoder.encoder.layer.0.output.dense.weight False
encoder.encoder.layer.

In [32]:
# from transformers import AutoTokenizer
# dec_tokenizer = AutoTokenizer.from_pretrained(
#     'bert-base-uncased'
# )
voc = dec_tokenizer.get_vocab()
voc["&quot;"]
# text = "we &apos; re going to be able to become god. &quot;"
# dec_tokenizer(text), dec_tokenizer.decode(dec_tokenizer(text)["input_ids"])

KeyError: '&quot;'

In [61]:
# model.add_adapter("testing")
# model.encoder.encoder.layer[0].output.adapters["testing"].adapter_down[0].weight
# model.decoder
model.config

EncoderDecoderConfig {
  "decoder": {
    "_name_or_path": "bert-base-uncased",
    "adapters": {
      "adapters": {},
      "config_map": {},
      "fusion_config_map": {},
      "fusions": {}
    },
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "is_decoder": true,

In [42]:
print(model.encoder.embeddings)
print(model.decoder.bert.embeddings)
# model.decoder.bert.encoder.layer[0].output.adapters["testing"].adapter_down[0].weight

BertEmbeddings(
  (word_embeddings): Embedding(31102, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [58]:
# model.encoder.add_adapter("testing")
# model.train_adapter("testing")
# model.active_adapters
for k, v in model.named_parameters():
    if "word_embeddings" in k:
        print(k, v.requires_grad)

encoder.embeddings.word_embeddings.weight False
decoder.bert.embeddings.word_embeddings.weight False


In [46]:
from transformers.trainer_utils import SchedulerType, get_last_checkpoint
from transformers.utils import check_min_version
from transformers.optimization import get_scheduler
import torch
import torch.nn as nn

model = nn.Linear(10, 10)
optimizer = torch.optim.Adam(model.parameters())

get_scheduler("linear", optimizer)

ValueError: SchedulerType.LINEAR requires `num_warmup_steps`, please provide that argument.

In [59]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("bert-base-uncased")
config

BertConfig {
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [70]:
lens = []
with open("../dataset/iwslt14/valid.de", "r") as f:
    for line in f.readlines():
        lens.append(len(line.strip().split(" ")))

lens1 = []
with open("../dataset/iwslt14/valid.en", "r") as f:
    for line in f.readlines():
        lens1.append(len(line.strip().split(" ")))

In [71]:
from scipy import stats
stats.describe(lens), stats.describe(lens1)

(DescribeResult(nobs=7283, minmax=(2, 142), mean=19.2823012494851, variance=177.182310990258, skewness=1.9489783183319207, kurtosis=6.457417276475706),
 DescribeResult(nobs=7283, minmax=(2, 143), mean=20.352190031580392, variance=193.40752987764301, skewness=1.862383613360634, kurtosis=5.389799944582457))

In [4]:
from transformers import Seq2SeqTrainer
Seq2SeqTrainer.create_scheduler

<function transformers.trainer.Trainer.create_scheduler(self, num_training_steps: int, optimizer: torch.optim.optimizer.Optimizer = None)>